In [4]:
// nuget references
#r "nuget: FSharp.Stats, 0.5.1-preview.1"
#r "nuget: Plotly.NET.Interactive, 4.2.1"
#r "nuget: FSharp.Data, 6.3.0"
#r "nuget: Cytoscape.NET.Interactive, 0.2.0"

open FSharp.Stats
open Plotly.NET
open Plotly.NET.StyleParam
open Plotly.NET.LayoutObjects
open FSharp.Data
open Cytoscape.NET
open System


//FSharp.Stats.ServiceLocator.setEnvironmentPathVariable (@"C:\Users\bvenn\source\repos\FSharp.Stats\lib")
//FSharp.Stats.Algebra.LinearAlgebra.Service()

//axis styling extension module
module Chart = 
    let myAxis name = LinearAxis.init(Title=Title.init name,Mirror=StyleParam.Mirror.All,Ticks=StyleParam.TickOptions.Inside,ShowGrid=false,ShowLine=true)
    let withAxisTitles x y chart = 
        chart 
        |> Chart.withTemplate ChartTemplates.lightMirrored
        |> Chart.withXAxis (myAxis x) 
        |> Chart.withYAxis (myAxis y)

type Category =
    | Beer
    | Beverage
    | Coffee
    | Other
    with 
        static member FromString (s: string) =
            match s with
            | "Beer" -> Beer
            | "Beverage" -> Beverage
            | "Coffee" -> Coffee
            | _ -> Other

type Order = {
    DateTime    : System.DateTime
    Name        : string
    Gender      : char
    Product     : string
    Price       : float
    Department  : string
    Category    : Category
    Amount      : int
    } with
        static member Create time (name: string) gender product price department category amount = {
            DateTime  = time
            Name      = name
            Gender    = gender
            Product   = product
            Price     = price
            Department= department
            Category  = category
            Amount    = amount
            }

let data = 
    let read =
        CsvFile
            //.Load(@"..\data\coffeedata.txt")
            .Load(@"C:\Users\bvenn\source\repos\brewing-discoveries-workshop\data\coffeedata.txt")
            .Cache()
    read.Rows
    |> Seq.map (fun row -> 
        Order.Create
            (System.DateTime.ParseExact((row.GetColumn "DateTime"),"dd/MM/yyyy HH:mm:ss",null))
            (row.GetColumn "Name")
            (row.GetColumn "Gender" |> char)
            (row.GetColumn "Product")
            (row.GetColumn "Price" |> float) 
            (row.GetColumn "Department")
            ((row.GetColumn "Category") |> Category.FromString)
            (row.GetColumn "Amount" |> int)
        )
    |> Array.ofSeq

let getDepartmentColor (department: string) = 
    match department with 
    | "Breakroom Bandits" -> "#2b3ae9"
    | "Genesis" -> "#f7da41"
    | "We Tried" -> "#008b66"
    | "No Lucks Given" -> "#987200"
    | "Toon Squad" -> "#ff7f0e"
    | "Rumor Spreaders" -> "#20b2aa"
    | "Risky Biscuits" -> "#a230ed"
    | "Recruitables" -> "#d21102"
    | "Employees of the Moment" -> "#19d3f3"
    | "Chargers" -> "#dea57b"
    | "Kickstarters" -> "#dea57b"
    | _ -> "#8b8b8b"

Installed Packages Cytoscape.NET.Interactive, 0.2.0 FSharp.Data, 6.3.0 FSharp.Stats, 0.5.1-preview.1 Plotly.NET.Interactive, 4.2.1

In [17]:
let persons = 
    data 
    |> Array.map (fun x -> x.Name,getDepartmentColor x.Department) 
    |> Array.distinct
    |> Map.ofArray

let dailyLogs = 
    data
    |> Array.groupBy (fun x -> x.Name)
    |> Array.map (fun (name,orders) -> 
        let tmp = 
            orders
            |> Array.groupBy (fun o -> o.DateTime.Date)
            |> Array.map (snd >> Array.length >> float)
        name,tmp)

[
    dailyLogs
    |> Array.filter (fun (name,data) -> data.Length > 10)
    |> Array.map (fun (name,data) -> 
        Chart.Point([Seq.mean data],[Seq.stDev data],Name=name,MarkerColor=Color.fromHex (persons.[name]))
        )
    |> Chart.combine
    |> Chart.withAxisTitles "mean(logs per day)" "stDev(logs per day)"


    dailyLogs
    |> Array.filter (fun (name,data) -> data.Length > 10)
    |> Array.map (fun (name,data) -> 
        let cv = Seq.cv data
        cv,Chart.Column([cv],[name],MarkerColor=Color.fromHex persons.[name],ShowLegend=false)
        )
    |> Array.sortBy fst
    |> Array.map snd
    |> Chart.combine
    |> Chart.withAxisTitles "" "coefficient of variation (mean/stdev)"
]
|> Chart.Grid(1,2)
|> Chart.withMarginSize(Bottom=150)
|> Chart.withSize(1200,600)

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let personsCorr = 
    persons
    |> Map.toArray
    |> Array.map fst

let signal = 
    data
    |> Array.groupBy (fun x -> x.DateTime.Date)
    |> Array.map (fun (date,orders) -> 
        
        let tmp = 
            personsCorr
            |> Array.map (fun name -> 
                orders
                |> Array.filter (fun order -> order.Name = name)
                |> Array.length
                |> float
                )
        tmp
        )
    |> JaggedArray.transpose
    
let pcapersoncs,pcaSignal = 
    Array.zip personsCorr signal
    |> Array.filter (fun (name,signal) -> 
        let days = signal.Length
        let ticking = signal |> Array.filter (fun x -> x > 0.) |> Array.length
        float ticking / float days > 0.05
        )
    |> Array.unzip

Chart.Heatmap(pcaSignal,colNames=(data|> Array.map (fun x -> x.DateTime.ToShortDateString()) |> Seq.distinct),rowNames=pcapersoncs)
|> Chart.withSize(1400.,900.)
|> Chart.show